# Keyness Metrics (for Lemmatized Corpora)

Setup...

In [ ]:
from src.corpus import Corpus, FrequencyCorpus
from src.metrics import keyness
from src.corpus_creation import document_retriever as dr
import pathlib
import json
import pandas as pd
import random
from src.load_data import load_files

%load_ext autoreload
%autoreload 2

Loading the data...

Put the path to your corpus in the variable `CORPUSDIR`.

I assume that the data is a set of json files, each containing a list of lemmata under the key 'lemmas'.
If you have a different format, you need to adjust the code accordingly. The result should be a list of lists of lemmata.

In [ ]:
# Put the path to the directory containing the corpus files here
CORPUSDIR = '/home//.../full'

docs, metadata = load_files(CORPUSDIR)
corpus = Corpus(docs, metadata)

Now, let's create two subcorpora:

1. STUDY CORPUS: All documents containing the word 'KI' or the 2-gram 'künstlich Intelligenz' at least once (You could of course also use different search terms or increase the number of hits needed to include a document.)
2. REFERENCE CORPUS: All other documents. (Here, you could also use a completely different corpus, e.g. Leipzig Corpora).

In [ ]:
hits = dr.match_wordlist(
    corpus, ['KI', ('künstlich', 'Intelligenz'),], min=1
)

study_corpus = dr.corpus_from_found(
    hits, source_corpus=corpus,
    goal_corpus='FrequencyCorpus'
)
reference_corpus = dr.corpus_from_notfound(
    hits, source_corpus=corpus,
    goal_corpus='FrequencyCorpus'
)
# with open('/home/.../Data/Reference/Leipzig-Corpora/reference_corpus_20-25.json', 'r') as f:
#     reference_corpus = json.load(f)
#     reference_corpus = random.sample(reference_corpus, 30000)
# reference_corpus = FrequencyCorpus(reference_corpus)

Now, for every word in our study corpus, we calculate its keyness score. You can use different metrics here:

1. Statistical Significance:
    + Log-Likelihood (according to Chi-Square)
    + Log-Ratio (according to Rayson)
2. Effect size:
    + Odds Ratio
    + Percentage Difference

In [ ]:
keynesses = keyness.keyword_list(
    study_corpus, reference_corpus,
    metric='percent_difference',
    min_docs=3,
    smoothing=0.5,
    max_ngram_len=1,
    filter_stopwords=True,
)

Filter the results with a minimum keyness and take only the top n results.

In [ ]:
from src.corpus_creation import handle_wordlists

filtered_df = handle_wordlists.remove_redundant(
    keynesses
)
filtered_df = filtered_df[filtered_df['Keyness'] > 2.0]


# top-50
filtered_df = filtered_df.head(50)
filtered_df

Now, we can use the list of keywords to create our thematic corpus...

In [ ]:
found_docs = dr.match_wordlist(
    corpus, filtered_df['Word'].tolist(), min=1
)
created_corpus = dr.corpus_from_found(
    found_docs, source_corpus=corpus,
    goal_corpus='Corpus'
)
